# ZeroShot分類

In [1]:
from llmclassifier import Classifier
import pandas as pd
from langchain.llms import OpenAI
import os

# llmの設定
llm = OpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])

# タスクの設定
task = """あなたはニュースのジャンル分類をしています。
ニュースのタイトルを読んで、そのジャンルを判断してください。"""

# ラベルの設定
classes = ["金融", "芸能", "食品", "音楽", "自動車", "経済", "政治", "スポーツ", "IT", "エンタメ", "科学", "国際", "地域", "健康", "教育", "お笑い", "事件", "その他"]

# 入力データの設定
inputs = pd.DataFrame([
        ["日本銀行が金利を引き下げる"],
        ["ユーチューバーMUKAKINが「みんな有料会員になって」と自身のチャンネルで発言し話題に"],
        ["雲印、新製品「体いきいきヨーグルト」発売。体内フローラを整える"],
        ["麦津犬歯、新曲「入れもん」MV公開。曲に合わせて入れ物に残った匂いを嗅ぐ内容"],
        ["TOYBOTAが大幅な方針転換を発表。全ての水上自動車を電気自動車にする。来年2024年までに切り替えを完了する"],
        ["三月の鉄鋼市況は、前月比で鉄鋼製品の生産量は前年同月比で1.5％減少し、鉄鋼製品の販売量は前年同月比で1.1％減少した。"],
    ],
    columns=["ニュースタイトル"])

# annotatorの設定
clsf = Classifier(llm=llm, task=task, classes=classes, multi_label=False)

In [2]:
# 結果の取得
y_pred = clsf.predict(inputs, return_wrapper=False)
outputs = clsf.predict(inputs, return_wrapper=True)
print(f"アノテーション結果のindex:\n{y_pred}")
display(pd.concat([inputs, outputs.get_df()], axis=1))

アノテーション結果のindex:
[0, 9, 13, 3, 4, 5]


,ニュースタイトル,index,label,str,source_node
0,日本銀行が金利を引き下げる,5,経済,経済,None
1,ユーチューバーMUKAKINが「みんな有料会員になって」と自身のチャンネルで発言し話題に,9,エンタメ,エンタメ,None
2,雲印、新製品「体いきいきヨーグルト」発売。体内フローラを整える,13,健康,健康,None
3,麦津犬歯、新曲「入れもん」MV公開。曲に合わせて入れ物に残った匂いを嗅ぐ内容,3,音楽,音楽,None
4,TOYBOTAが大幅な方針転換を発表。全ての水上自動車を電気自動車にする。来年2024年まで...,4,自動車,自動車,None
5,三月の鉄鋼市況は、前月比で鉄鋼製品の生産量は前年同月比で1.5％減少し、鉄鋼製品の販売量は前...,5,経済,経済,None


# FewShot分類

In [2]:
X_train = pd.DataFrame([
        ["日銀が引き続き金利の引き上げを行うことを表明。"],
        ["ユーチューバー柴田が「世界のみんなを助けたい」と世界平和を訴える"],
        ["乳酸菌飲料の睡眠効果を調査。統計的に有意なことが確認された"],
        ["放課後音楽クラブがゲーム「パチモン」のED「パチモンしりとり」を発表"],
        ["MISSANが全ての自動車にリチウムバッテリーを搭載すると発表"],
        ["供給不足が解消しガソリン市場急落。前月比5%の値下がりを記録"],
    ],
    columns=["ニュースタイトル"])
y_train = ["政治", "お笑い", "地域", "国際", "事件", "その他"]

In [3]:
clsf.fit(X_train, y_train)
outputs = clsf.predict(inputs, return_wrapper=True)
display(pd.concat([inputs, outputs.get_df()], axis=1))

,ニュースタイトル,index,label,str,source_node
0,日本銀行が金利を引き下げる,0,金融,金融,[Q_ex:ニュースタイトル:日銀が引き続き金利の引き上げを行うことを表明。\nA_ex:政...
1,ユーチューバーMUKAKINが「みんな有料会員になって」と自身のチャンネルで発言し話題に,9,エンタメ,エンタメ,[Q_ex:ニュースタイトル:日銀が引き続き金利の引き上げを行うことを表明。\nA_ex:政...
2,雲印、新製品「体いきいきヨーグルト」発売。体内フローラを整える,2,食品,雲印、新製品「体いきいきヨーグルト」発売。体内フローラを整える\na:食品,[Q_ex:ニュースタイトル:日銀が引き続き金利の引き上げを行うことを表明。\nA_ex:政...
3,麦津犬歯、新曲「入れもん」MV公開。曲に合わせて入れ物に残った匂いを嗅ぐ内容,9,エンタメ,エンタメ,[Q_ex:ニュースタイトル:日銀が引き続き金利の引き上げを行うことを表明。\nA_ex:政...
4,TOYBOTAが大幅な方針転換を発表。全ての水上自動車を電気自動車にする。来年2024年まで...,4,自動車,自動車,[Q_ex:ニュースタイトル:日銀が引き続き金利の引き上げを行うことを表明。\nA_ex:政...
5,三月の鉄鋼市況は、前月比で鉄鋼製品の生産量は前年同月比で1.5％減少し、鉄鋼製品の販売量は前...,0,金融,金融,[Q_ex:ニュースタイトル:日銀が引き続き金利の引き上げを行うことを表明。\nA_ex:政...


# LlamaIndexをつかった分類

In [3]:
clsf.fit(X_train, y_train, use_index=True)
outputs = clsf.predict(inputs, return_wrapper=True)
display(pd.concat([inputs, outputs.get_df()], axis=1))

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 409 tokens
INFO:root:> [query] Total LLM token usage: 452 tokens
INFO:root:> [query] Total embedding token usage: 232 tokens
INFO:root:> [query] Total LLM token usage: 499 tokens
INFO:root:> [query] Total embedding token usage: 268 tokens
INFO:root:> [query] Total LLM token usage: 475 tokens
INFO:root:> [query] Total embedding token usage: 255 tokens
INFO:root:> [query] Total LLM token usage: 503 tokens
INFO:root:> [query] Total embedding token usage: 275 tokens
INFO:root:> [query] Total LLM token usage: 512 tokens
INFO:root:> [query] Total embedding token usage: 289 tokens
INFO:root:> [query] Total LLM token usage: 558 tokens
INFO:root:> [query] Total embedding token usage: 318 tokens


,ニュースタイトル,index,label,str,source_node
0,日本銀行が金利を引き下げる,0,金融,金融,[SourceNode(source_text='Q_ex:ニュースタイトル:日銀が引き続き...
1,ユーチューバーMUKAKINが「みんな有料会員になって」と自身のチャンネルで発言し話題に,9,エンタメ,エンタメ,[SourceNode(source_text='Q_ex:ニュースタイトル:ユーチューバー...
2,雲印、新製品「体いきいきヨーグルト」発売。体内フローラを整える,2,食品,食品,[SourceNode(source_text='Q_ex:ニュースタイトル:日銀が引き続き...
3,麦津犬歯、新曲「入れもん」MV公開。曲に合わせて入れ物に残った匂いを嗅ぐ内容,3,音楽,音楽,[SourceNode(source_text='Q_ex:ニュースタイトル:放課後音楽クラ...
4,TOYBOTAが大幅な方針転換を発表。全ての水上自動車を電気自動車にする。来年2024年まで...,4,自動車,自動車,[SourceNode(source_text='Q_ex:ニュースタイトル:missanが...
5,三月の鉄鋼市況は、前月比で鉄鋼製品の生産量は前年同月比で1.5％減少し、鉄鋼製品の販売量は前...,5,経済,経済,[SourceNode(source_text='Q_ex:ニュースタイトル:供給不足が解消...
